# Candlestick strategy - Backtesting

### Import Library

In [9]:
import numpy as np
import pandas as pd
import numpy as np
import time
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import requests
import pandas_ta as ta
import backtesting
backtesting.set_bokeh_output(notebook=False)
from backtesting.backtesting import Backtest, Strategy
import candlestick.price_action as pa

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [10]:
USER_AGENTS = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36"
]

HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
VNDIRECT_DATA_HISTORY_URL = 'https://dchart-api.vndirect.com.vn/dchart/history'
URL = VNDIRECT_DATA_HISTORY_URL

In [11]:
def getStockHistoryData(ticker, timestamp_from=0, timestamp_to=0):
    if timestamp_from == 0:
        three_months = date.today() + relativedelta(months=-3)
        timestamp_from = datetime.strptime(three_months.strftime("%m/%d/%Y") + ', 00:00:0', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()
    if timestamp_to == 0:
        timestamp_to = datetime.strptime(date.today().strftime("%m/%d/%Y") + ', 23:59:00', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()

    params = {
        "resolution": "5",
        "symbol": str(ticker),
        "from": int(timestamp_from),
        "to": int(timestamp_to)
    }

    x = requests.get(URL, params=params, headers=HEADERS)
    response = x.json()

    import numpy as np
    import pandas as pd

    timestamp = np.array(response['t']).astype(int)
    close = np.array(response['c']).astype(float)
    open = np.array(response['o']).astype(float)
    high = np.array(response['h']).astype(float)
    low = np.array(response['l']).astype(float)
    volume = np.array(response['v']).astype(int)

    dataset = pd.DataFrame({'Time': timestamp, 'Open': list(open), 'High': list(high), 'Low': list(low),
                            'Close': list(close), 'Volume': list(volume)},
                           columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    return dataset

In [12]:
ticker = "VN30F1M"
dataset = getStockHistoryData(ticker, 0, 0)

In [13]:
data = dataset.copy()

In [14]:
def prepare_data(data):
    if 'Time' in data.columns:
        from datetime import datetime

        data['DateStr'] = data.apply(
            lambda x: datetime.fromtimestamp(x['Time']).strftime("%Y-%m-%d %H:%M:%S"), axis=1)

    data['Date'] = pd.to_datetime(data['DateStr'])
    data = data.set_index('Date')
    data["current"] = data.index + pd.DateOffset(minutes=5)
    data["ema_f1"] = ta.ema(data["Close"], length=5)
    data["ema_l1"] = ta.ema(data["Close"], length=26)
    data["short_trend"] = data.apply(lambda r: TREND_UP if r['ema_f1'] > r['ema_l1'] else TREND_DOWN, axis=1)
    data["ema_f2"] = ta.ema(data["Close"], length=16)
    data["ema_l2"] = ta.ema(data["Close"], length=250)
    data["long_trend"] = data.apply(lambda r: TREND_UP if r['ema_f2'] > r['ema_l2'] else TREND_DOWN, axis=1)
    data.drop(columns=['Time', 'DateStr', 'ema_f1', 'ema_l1', 'ema_f2', 'ema_l2'], inplace=True)
    data.dropna(inplace=True)
    return data

In [16]:
prepared_data = prepare_data(data)

In [17]:
prepared_data = pa.prepare_data(prepared_data)

In [18]:
prepared_data

,Open,High,Low,Close,Volume,volume_20,ema_fast,ema_long,current,min_OC,max_OC,upper_wick,tail,oc_dif,body,color,candlestick
Date,,,,,,,,,,,,,,,,,
2024-06-05 11:05:00,1302.5,1303.5,1302.3,1303.3,3238,3127.20,1304.610647,1305.907692,2024-06-05 11:10:00,1302.5,1303.3,0.2,0.2,0.8,0.8,white,
2024-06-05 11:10:00,1303.0,1303.5,1302.5,1303.2,2620,2819.45,1304.328518,1305.707123,2024-06-05 11:15:00,1303.0,1303.2,0.3,0.5,0.2,0.2,white,
2024-06-05 11:15:00,1303.2,1303.8,1302.1,1303.8,3321,2713.15,1304.222814,1305.565854,2024-06-05 11:20:00,1303.2,1303.8,0.0,1.1,0.6,0.6,white,
2024-06-05 11:20:00,1303.7,1303.7,1302.8,1303.3,1944,2682.15,1304.038251,1305.398013,2024-06-05 11:25:00,1303.3,1303.7,0.0,0.5,-0.4,0.4,black,
2024-06-05 11:25:00,1303.4,1304.6,1303.3,1304.4,3106,2692.85,1304.110601,1305.324086,2024-06-05 11:30:00,1303.4,1304.4,0.2,0.1,1.0,1.0,white,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-05 10:25:00,1318.1,1318.6,1317.3,1317.3,4037,2863.00,1317.245520,1315.807711,2024-09-05 10:30:00,1317.3,1318.1,0.5,0.0,-0.8,0.8,black,
2024-09-05 10:30:00,1317.3,1317.4,1316.7,1316.7,2374,2979.70,1317.136416,1315.873807,2024-09-05 10:35:00,1316.7,1317.3,0.1,0.0,-0.6,0.6,black,
2024-09-05 10:35:00,1316.7,1317.0,1316.6,1316.7,1667,2679.05,1317.049133,1315.935006,2024-09-05 10:40:00,1316.7,1316.7,0.3,0.1,0.0,0.0,doji,doji


In [19]:
from candlestick import DOJI_CS, WHITE_CS, BLACK_CS
from candlestick import INVERTED_HAMMER_CS, SPINNING_TOP_CS, MARUBOZU_CS, HAMMER_CS, HANGING_MAN_CS, SHOOTING_STAR_CS

In [29]:
class VolumeVsEma(Strategy):
    def init(self):
        return

    def next(self):
        whole_data = self.data.df
        close_price = self.data.Close
        signal_data = whole_data.iloc[-1]
        current_time = signal_data['current']
        
        if current_time.hour == 14 and current_time.minute >= 27:
            if self.position.is_long or self.position.is_short:
                self.position.close()
            return

        if signal_data['long_trend'] == TREND_UP and signal_data['short_trend'] == TREND_DOWN:
            if signal_data['Close']

        if signal_data['Close'] > signal_data['Open']:
            if signal_data['Volume'] > signal_data['volume_20']:
                if self.position.is_short:
                    self.position.close()
                else:
                    self.buy()
        elif signal_data['Close'] < signal_data['Open']:
            if signal_data['Volume'] > signal_data['volume_20']:
                if self.position.is_long:
                    self.position.close()
                else:
                    self.buy()

In [30]:
bt = Backtest(prepared_data, VolumeVsEma, commission=.003, exclusive_orders=False)
stats = bt.run()

In [31]:
stats

Start                     2024-06-05 11:05:00
End                       2024-09-05 10:15:00
Duration                     91 days 23:10:00
Exposure Time [%]                   70.844854
Equity Final [$]                    3223.3089
Equity Peak [$]                       10000.0
Return [%]                         -67.766911
Buy & Hold Return [%]                1.089542
Return (Ann.) [%]                   -98.65435
Volatility (Ann.) [%]                0.174944
Sharpe Ratio                      -563.920963
Sortino Ratio                       -3.282377
Calmar Ratio                        -1.454888
Max. Drawdown [%]                  -67.808911
Avg. Drawdown [%]                  -67.808911
Max. Drawdown Duration       91 days 23:00:00
Avg. Drawdown Duration       91 days 23:00:00
# Trades                                  431
Win Rate [%]                         6.264501
Best Trade [%]                       0.990576
Worst Trade [%]                     -1.559986
Avg. Trade [%]                    

In [32]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,7,3,7,1307.6111,1304.3,-23.1777,-0.002532,2024-06-05 11:20:00,2024-06-05 13:05:00,None,0 days 01:45:00
1,7,9,10,1308.5138,1304.2,-30.1966,-0.003297,2024-06-05 13:15:00,2024-06-05 13:20:00,None,0 days 00:05:00
2,7,12,14,1307.7114,1302.1,-39.2798,-0.004291,2024-06-05 13:30:00,2024-06-05 13:40:00,None,0 days 00:10:00
3,7,15,16,1306.2069,1302.0,-29.4483,-0.003221,2024-06-05 13:45:00,2024-06-05 13:50:00,None,0 days 00:05:00
4,7,17,20,1308.1126,1304.0,-28.7882,-0.003144,2024-06-05 13:55:00,2024-06-05 14:10:00,None,0 days 00:15:00
...,...,...,...,...,...,...,...,...,...,...,...
426,2,3191,3193,1316.0363,1312.2,-7.6726,-0.002915,2024-09-04 09:15:00,2024-09-04 09:25:00,None,0 days 00:10:00
427,2,3212,3223,1313.4285,1310.0,-6.8570,-0.002610,2024-09-04 11:00:00,2024-09-04 13:20:00,None,0 days 02:20:00
428,2,3226,3230,1314.9330,1309.8,-10.2660,-0.003904,2024-09-04 13:35:00,2024-09-04 13:55:00,None,0 days 00:20:00
429,2,3232,3236,1314.1306,1312.7,-2.8612,-0.001089,2024-09-04 14:05:00,2024-09-04 14:25:00,None,0 days 00:20:00
